### 1. Augmentation된 보라색 원 이미지 데이터를 학습하여 최적의 threshold 값 추출

In [3]:
import cv2 as cv
import numpy as np
from sklearn.cluster import KMeans
import glob
import os

In [9]:
# 보라색 원이 포함된 augmentation된 이미지들 경로 설정
img_folder = 'output_folder/*.jpg'
img_paths = glob.glob(img_folder)

In [10]:
# 이미지 전처리 함수
def process_image(img_path):
    # 이미지 로드 및 흑백 변환 (명도값 활용 예정)
    img = cv.imread(img_path)
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # 이미지를 RGB 3차원에서 1차원 배열로 변환
    pixel_values = img_gray.reshape((-1, 1))
    pixel_values = np.float32(pixel_values)

    return pixel_values, img.shape

In [11]:
# 모든 이미지에서 pixel data 수집해서 K-means 학습용 dataset 생성
def create_training_data(img_paths):
    training_data = []

    for img_path in img_paths:
        pixel_values, _ = process_image(img_path)
        training_data.append(pixel_values)
    
    # 데이터를 하나로 합치기!!
    training_data = np.vstack(training_data)

    return training_data

In [12]:
training_data = create_training_data(img_paths)

In [13]:
# K-means clustring을 통한 임계값 추출
def kmeans_threshold(training_data, n_clusters=2):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)

    kmeans.fit(training_data)

    # 각 클러스터의 중심값 게산
    centers = kmeans.cluster_centers_

    # 중심값 중 더 높은 값은 threshold로 사용
    # 보라색 pixel 값들을 대표하는 거임!!
    threshold = max(centers)
    print(f"추출된 최적의 threshold 값 : {threshold[0]}")

    return threshold[0]

In [14]:
threshold = kmeans_threshold(training_data)